In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *

spark = SparkSession.builder.enableHiveSupport().getOrCreate()

In [2]:
# ! hdfs dfs -ls /data/stackoverflow/parquet

In [3]:
# spark.catalog.createTable('stackoverflow.tags', path='/data/stackoverflow/parquet/Tags')

In [4]:
DAY = '2019-09-01'

In [5]:
posts = spark.table('stackoverflow.posts')

In [6]:
posts.groupBy('PostTypeId').count().show()

+----------+--------+
|PostTypeId|   count|
+----------+--------+
|         1|18154493|
|         6|     324|
|         3|     167|
|         5|   49909|
|         4|   49909|
|         8|       2|
|         7|       4|
|         2|27665009|
+----------+--------+



In [7]:
is_question = (col('PostTypeId') == 1) # according to https://ia800107.us.archive.org/27/items/stackexchange/readme.txt
tag_popularity_in_questions = posts.where(is_question & (col('CreationDate').cast('date') == DAY)) \
    .select('Id', explode('Tags').alias('tag')) \
    .groupBy('tag').count()

In [8]:
# tag_popularity_in_questions.orderBy(desc('count')).show()

In [9]:
is_answer = (col('PostTypeId') == 2)

In [19]:
answers_today = posts.where(is_answer & (col('CreationDate').cast('date') == DAY)).alias('a')
all_questions = posts.where(is_question).alias('q')
tag_popularity_in_answers = answers_today.join(all_questions, col('a.ParentId') == col('q.Id')).select('q.Tags') \
    .select(explode('Tags').alias('tag')).groupBy('tag').count()